Notebook to calculate CAPE using Daniel Gilford's CAPE code for Rosi's MPAS aquaplanet grid. Outputs in Zarr format per time step for entire grid.

James Ruppert  
9/1/2025

### Main settings

In [1]:
from numba import njit, prange
import pyPI_main
import numpy as np
import pickle
import xarray as xr
# from precip_class_mpas import *
import os, shutil
# import matplotlib.pyplot as plt
# from matplotlib import colors
# import cartopy.crs as ccrs

In [2]:
do_write = True
# do_write = False

pclass_names = ['DC', 'CG', 'SC', 'ST', 'AN', 'DSA']
nclass = len(pclass_names)
exp_names = ["CTL", "HOMO_RAD", "CLIM_RAD"]
expName=exp_names[0]

pickle_dir = f"/glade/derecho/scratch/ruppert/tc-crfrad/pickle_out"
# pickle_dir = f"../../../pickle_out/aquaplanet"

# Get list of desired file times
file_times_arr = np.arange('2000-05-01T06:00:00', '2000-05-11T06:00:00', 6, dtype='datetime64[h]')
file_times = [file_times_arr[i].astype('datetime64[D]').astype(str)+'_'+str(file_times_arr[i]).split('T')[1].split(':')[0]+'.00.00' for i in range(len(file_times_arr))]
ntimes = len(file_times)

#### Start cluster and scale it up

In [3]:
# from ncar_jobqueue import NCARCluster
# cluster = PBSCluster()
# cluster.scale(4 * 9) # Ask for 4 x 9 workers
# cluster

from dask_jobqueue import PBSCluster
# Create a PBS cluster object
# cluster = PBSCluster(
#     job_name = 'dask-wk23-hpc',
#     cores = 1,
#     processes = 1,
#     log_directory = '/glade/derecho/scratch/rberrios/dask/',
#     local_directory = '/glade/derecho/scratch/rberrios/dask/',
#     resource_spec = 'select=1:ncpus=1:mem=10GB',
#     queue = 'casper',
#     walltime = '12:00:00',
#     interface = 'mgt'
# )
cluster = PBSCluster(
    job_name = 'dask-wk23-hpc',
    cores = 1,
    processes = 1,
    memory = '10GiB',
    account = 'UOKL0049',
    log_directory = '/glade/derecho/scratch/ruppert/dask/',
    local_directory = '/glade/derecho/scratch/ruppert/dask/',
    resource_spec = 'select=1:ncpus=1:mem=10GB',
    queue = 'casper',
    # queue = 'main',
    walltime = '12:00:00',
    interface = 'ext'
)
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-wk23-hpc
#PBS -q casper
#PBS -A UOKL0049
#PBS -l select=1:ncpus=1:mem=10GB
#PBS -l walltime=12:00:00
#PBS -e /glade/derecho/scratch/ruppert/dask//
#PBS -o /glade/derecho/scratch/ruppert/dask//

/glade/work/ruppert/conda-envs/plotting/bin/python -m distributed.cli.dask_worker tcp://128.117.211.222:38637 --name dummy-name --nthreads 1 --memory-limit 10.00GiB --nanny --death-timeout 60 --local-directory /glade/derecho/scratch/ruppert/dask/ --interface ext



In [4]:
from dask.distributed import Client
client = Client(cluster) # Connect this local process to remote workers
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://128.117.211.222:8787/status,
Dashboard: http://128.117.211.222:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.211.222:38637,Workers: 0
Dashboard: http://128.117.211.222:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
# Scale the cluster to n workers (which will use n jobs here)
ncpu = 36
# ncpu = 24
# ncpu = 10
# ncpu = 18
cluster.scale(ncpu)

# Block progress until workers have spawned (typically only in demos and benchmarks!)
client.wait_for_workers(ncpu)

### Read/Write variable from MPAS output

In [6]:
# Read grid data
do_write_grid = False
grid_pickle_file = f"{pickle_dir}/grid_data.pickle"
if do_write_grid:
    import xarray as xr
    grid_path = "/glade/work/rberrios/MPAS/aqua_sstmax10N_ASD/plus4K/TC_3km/x5.tropical_3km_10N.init.nc"
    grid = xr.open_dataset(grid_path)
    areaCell = grid.areaCell
    latCell = np.degrees(grid.latCell)
    lonCell = np.degrees(grid.lonCell)
    with open(grid_pickle_file, 'wb') as f:
        pickle.dump([areaCell, latCell, lonCell], f)
else:
    with open(grid_pickle_file, 'rb') as f:
        areaCell, latCell, lonCell = pickle.load(f)

#### CAPE/CIN function

In [7]:
def tmpk_from_theta(theta, pres):
    # theta in K, pres in Pa
    rd=287.04 # J/K/kg
    cp=1004. # J/K/kg
    rocp = rd/cp
    return theta * ( 1e5 / pres ) ** (-rocp)

"""
    Args:
        TP (float): Parcel temperature (K)
        RP (float): Parcel mixing ratio (gram/gram)
        PP (float): Parcel pressure (hPa)
        T (array): Environmental temperature profile (K)
        R (array): Environmental mixing ratio profile (gram/gram)
        P (array): Environmental pressure profile (hPa)
            The arrays MUST be arranged so that the lowest index corresponds
            to the lowest model level, with increasing index corresponding to
            decreasing pressure.
    Returns:
        tuple: (CAPED, TOB, LNB, IFLAG) where:
            - CAPED (float): Convective Available Potential Energy (J/kg)
            - TOB (float): Temperature at level of neutral buoyancy (K)
            - LNB (float): Level of neutral buoyancy pressure (hPa)
            - IFLAG (int): Status flag where:
                1 = Success
                0 = Improper sounding/parcel
                2 = Did not converge
                3 = Missing values in input profile
"""

@njit(parallel=True)
def cape_driver(T, q, p):

    ncol = T.shape[0]
    cape_out = np.empty(ncol, dtype=np.float64)
    cin_out  = np.empty(ncol, dtype=np.float64)

    for i in prange(ncol):
    # for i in range(ncol):
        (CAPE,CIN,TOB,LNB,IFLAG)=pyPI_main.cape(T[i,0],q[i,0],p[i,0]*1e-2,
                                                T[i,:],q[i,:],p[i,:]*1e-2,
                                                ascent_flag=0,ptop=50,miss_handle=0)
        cape_out[i] = CAPE
        cin_out[i]  = CIN
    return cape_out, cin_out

def cape_wrapper(T, q, p):
    # expects numpy arrays (Dask will pass chunks as np)
    return cape_driver(T, q, p)

#### Time loop

In [8]:
# Possible looping over time

# if do_write:

model_path = "/glade/campaign/mmm/dpm/rberrios/glade_scratch/MPAS_APE/aqua_sstmax10N_ASD/"

for expName in exp_names:

    writeout_path = f"/glade/derecho/scratch/ruppert/tc-crfrad/mpas/{expName}"

    for it in range(ntimes):
    # for it in range(19,20):

        time = file_times[it]

        out_path = f"{writeout_path}/CAPE_{time}.zarr"
        if os.path.exists(out_path):
            # shutil.rmtree(out_path)
            print(f"{out_path} exists, skipping...")
            continue

        # Choose experiment and read in model data
        data_path = f"{model_path}{expName}/TC_3km/"
        diag_files = data_path+'diag.'+time+'.nc'

        # nCells_chunk_size = 100000
        nCells_chunk_size = 10000
        ds = xr.open_mfdataset(diag_files,
                    parallel=True, 
                    chunks={"Time": -1, "nCells": nCells_chunk_size})

        ds = ds.isel(Time=0)#,  nCells=slice(0,10000))
        # ds = ds.isel(nCells=slice(0,100000))

        # apply function over chunks
        cape, cin = xr.apply_ufunc(
            cape_wrapper,
            tmpk_from_theta(ds.theta, ds.pressure), ds.qv, ds.pressure,
            input_core_dims=[["nVertLevels"], ["nVertLevels"], ["nVertLevels"]],
            output_core_dims=[[], []],     # scalar per column for both outputs
            vectorize=False,
            dask="parallelized",
            output_dtypes=[float, float],
        )

        cape.name = "CAPE"
        cin.name = "CIN"

        # Variables attributes
        cape.attrs = {
            "long_name": "Surface parcel CAPE",
            "units": "J kg-1",
            "description": "Computed using pyPI functions from Daniel Gilford"
        }
        cin.attrs = {
            "long_name": "Surface parcel CINH",
            "units": "J kg-1",
            "description": "Negative buoyant energy before reaching LFC"
        }

        merged = xr.merge([cape, cin])

        # out_path = f"{writeout_path}/CAPE_{time}.nc"
        # merged.to_netcdf(out_path, mode="w", compute=True, format='NETCDF4')

        # Write to Zarr; Dask computes in chunks
        merged.to_zarr(out_path, mode='w', compute=True,
                    encoding = {
                        "CAPE": {"dtype": "float32", "chunks": (nCells_chunk_size,), "compressor": None},
                        "CIN":  {"dtype": "float32", "chunks": (nCells_chunk_size,), "compressor": None},
                    })

        print(f"Done writing {out_path}!")

/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-01_06.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-01_12.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-01_18.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-02_00.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-02_06.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-02_12.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-02_18.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-03_00.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-03_06.00.00.zarr exists, skipping...
/glade/derecho/scratch/ruppert/tc-crfrad/mpas/CTL/CAPE_2000-05-03_12.00.00.zarr exists, skipping...


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/core/group.py:2566: ZarrUserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/api/asynchronous.py:233: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 19.78 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Done writing /glade/derecho/scratch/ruppert/tc-crfrad/mpas/HOMO_RAD/CAPE_2000-05-01_06.00.00.zarr!


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/core/group.py:2566: ZarrUserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/api/asynchronous.py:233: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 19.78 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Done writing /glade/derecho/scratch/ruppert/tc-crfrad/mpas/HOMO_RAD/CAPE_2000-05-01_12.00.00.zarr!


...

Done writing /glade/derecho/scratch/ruppert/tc-crfrad/mpas/CLIM_RAD/CAPE_2000-05-10_12.00.00.zarr!


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/core/group.py:2566: ZarrUserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/api/asynchronous.py:233: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 19.79 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Done writing /glade/derecho/scratch/ruppert/tc-crfrad/mpas/CLIM_RAD/CAPE_2000-05-10_18.00.00.zarr!


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/core/group.py:2566: ZarrUserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/zarr/api/asynchronous.py:233: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 19.79 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Done writing /glade/derecho/scratch/ruppert/tc-crfrad/mpas/CLIM_RAD/CAPE_2000-05-11_00.00.00.zarr!


### Close cluster

In [9]:
cluster.close()

### Plot CAPE

In [10]:
# ll_bounds = (0,60,0,30)
# filter = np.where((lonCell >= ll_bounds[0]) &
#                   (lonCell <= ll_bounds[1]) &
#                   (latCell >= ll_bounds[2]) &
#                   (latCell <= ll_bounds[3]))[0]

In [11]:
# sns.set_palette('muted')

# # for it, itime in enumerate(file_times_read):

# fig = plt.figure(figsize=(12,10))
# ax = fig.add_subplot(111, projection=ccrs.Mollweide())
# ax.gridlines()#draw_labels=True, x_inline=False, y_inline=False)#, dms=True)

# sc = ax.scatter(lonCell, latCell, c=cape, s=1, transform=ccrs.PlateCarree())

# cbar=plt.colorbar(sc, ax=ax, fraction=0.03, pad=0.02, shrink=0.65)
# ax.set_title(f"CAPE ({expName}, {time})", fontsize=16)
# # plt.ylim((-5,25))

# plt.tight_layout()
# plt.show()
# # plt.savefig(f"cape_map_{expName}_{time}.png", dpi=200)